# Load packages
* Load ImpedanceFitter and Matplotlib for the visualization
* Adjust the figure size because otherwise the plots are relatively small

In [ ]:
import impedancefitter as ifit
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = [15, 10]

# Load data

* Available file formats are listed here: https://impedancefitter.readthedocs.io/en/latest/examples/fileformats.html
* We use two CSV files provided by Henning Bathel

In [ ]:
fitter = ifit.Fitter("CSV")

The data is stored in dictionaries.
There is the `omega_dict` saving the frequencies.
It comprises only one list of frequencies per file.

In [ ]:
print(fitter.omega_dict.keys())
print(fitter.omega_dict['impedance1V_10.csv'].shape)

The impedances are saved in the `z_dict`.
It comprises a nested list containing multiple impedance lists.
This is because multiple recordings can be read for the same frequencies from one file.

In [ ]:
print(fitter.z_dict.keys())
print(fitter.z_dict['impedance1V_10.csv'].shape)

# Plot the data

Show all files separately or enable the `allinone=True` option.
You can also check other plottypes, for example, a Bode plot.

In [ ]:
fitter.visualize_data()

In [ ]:
fitter.visualize_data(allinone=True)

In [ ]:
fitter.visualize_data(allinone=True, plottype="bode")

# Check the data validity

* The LinKK test is used
* Here, we have relatively noisy data with strong capacitive behaviour at low frequencies
* To see the error, we adjust the plot limits for the relative difference to +- 15%
* To check the validity, we subsequently add a capacitance and inductance to the LinKK test

In [ ]:
results, mus, residuals = fitter.linkk_test(limits=[-15 ,15])

In [ ]:
results, mus, residuals = fitter.linkk_test(capacitance=True, limits=[-15 ,15])

In [ ]:
results, mus, residuals = fitter.linkk_test(capacitance=True, inductance=True, limits=[-15 ,15])

In [ ]:
_, _, _ = fitter.linkk_test(capacitance=True, inductance=True, limits=[-10 ,10], c=0.5)

# Choosing a proper frequency range

* The observed residual hints at valid data between $10^4$ and $5\times10^6$ Hz and possibly a lead inductance
* This hypothesis is in the following confirmed

In [ ]:
fitter = ifit.Fitter("CSV", minimumFrequency=1e4, maximumFrequency=5e6)

In [ ]:
_, _, _ = fitter.linkk_test(capacitance=True, limits=[-10 ,10], c=0.5)

In [ ]:
_, _, _ = fitter.linkk_test(capacitance=True, inductance=True, limits=[-10 ,10], c=0.5)

# Fit the data

* The hypothesis is that the model comprises a CPE in series with a resistance and an inductance
* A corresponding model is formulated
* A dictionary with initial guesses for the parameters is prepared

In [ ]:
model = "CPE + R + L"

In [ ]:
parameters = {"R": {"value": 55},
              "k": {"value": 100},
              "alpha": {"value": 0.8, "min": 0.6, "max": 0.9},
              "L": {"value": 1e-9}
             }

# Run the fit

**Note**: 
* The `residual="absolute` option is chosen because the imaginary part tends to zero at higher frequencies.
* The `weighting="modulus"` option is chosen to equilibrate the error over the entire frequency range. Different weighting schemes are discussed in relevant textbooks.
* Observe the fit report regarding the errors and correlations.

In [ ]:
fitter.run(model, parameters=parameters, report=True, show=True, residual="absolute", weighting="modulus")

# Postprocess the fit result

* Define submodels of the actual model
* Generate equivalent circuits to compute the impedance of the submodel
* Use the fit result stored in the `fit_data` dictionary
* The results are compared in different plots (should be self-explanatory)

In [ ]:
modelR = "R"
modelCPE = "CPE"
modelLR = "R + L"

In [ ]:
ecmR = ifit.get_equivalent_circuit_model(modelR)
ecmCPE = ifit.get_equivalent_circuit_model(modelCPE)
ecmLR = ifit.get_equivalent_circuit_model(modelLR)

In [ ]:
omega = fitter.omega_dict["impedance1V_10.csv"]

In [ ]:
ZR = ecmR.eval(omega=omega, **fitter.fit_data["impedance1V_10.csv_0"])
ZCPE = ecmCPE.eval(omega=omega, **fitter.fit_data["impedance1V_10.csv_0"])
ZLR = ecmLR.eval(omega=omega, **fitter.fit_data["impedance1V_10.csv_0"])

In [ ]:
ifit.plot_impedance(omega, ZR, labels=["R", "", ""], append=True, show=False)
ifit.plot_impedance(omega, ZCPE, labels=["CPE", "", ""], append=True, show=False)
ifit.plot_impedance(omega, ZCPE + ZR, labels=["CPE + R", "", ""], append=True, show=False)
ifit.plot_impedance(omega, ZLR, labels=["LR", "", ""])

In [ ]:
ifit.plot_impedance(omega, ZR, Z_comp=ZLR, labels=["R", "", "LR"])